In [1]:
#Import dependencies
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from flask import Flask, render_template, redirect

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\ljean\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
############ PART 1 - NASA MARS NEWS  ##############################

#Establish Mars News URL to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html
response = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [ ]:
#pull in results to enable easier parsing
results = soup.find_all('div', class_='content_title')
results

In [4]:
# Return the latest news title and paragraph stored as variables
news_title = soup.find_all('div', class_='content_title')[1].text
news_p = soup.find_all('div', class_='article_teaser_body')[0].text
print('----------------------------')
print(news_title)
print('------------------------------------------------------------------------')
print(news_p)
print('----------------------------')

----------------------------
New Study Challenges Long-Held Theory of Fate of Mars' Water
------------------------------------------------------------------------
The new science results indicate that a large quantity of the Red Planet’s water is trapped in its crust rather than having escaped into space.
----------------------------


In [ ]:
############ PART 2 - JPL Mars Space Images  ##############################

In [5]:
#Use splinter to navigate the site and find the image url for the current Featured Mars Image 
#and assign the url string to a variable called featured_image_url.

#establish base url link
jpl_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(jpl_url)
jpm_html = browser.html
response = requests.get(jpl_url)

# Create BeautifulSoup object; parse with 'html.parser'
soup_image = bs(jpm_html, 'html.parser')

In [21]:
image = soup_image.find_all('img', class_='headerimage fade-in')
image

[<img class="headerimage fade-in" src="image/featured/mars3.jpg"/>]

In [ ]:
#Find the image url to the full size .jpg image.
full_image_url = 


In [ ]:
#Make sure to save a complete url string for this image.